In [3]:
%pip install matplotlib seaborn plotly plotnine

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:

# %matplotlib widget
import matplotlib.pyplot as plt
from polygonal_roadmaps import cli, environment
import networkx as nx
import numpy as np
import logging
import pandas as pd
import seaborn as sns
from IPython.display import HTML
import plotly.express as px
import plotly.graph_objects as go
import tempfile
import plotly.express as px

logging.getLogger().setLevel(logging.INFO)

%load_ext snakeviz

In [5]:
scenarios = ["MAPF;room-32-32-4;even", "MAPF;empty-8-8.map;even", "DrivingSwarm;icra2024.yaml;icra2024_1m.yml"]
env = cli.env_generator(scenarios[-1], n_agents=4)[0]
env.state


/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.shared_paths(a, b, **kwargs)
/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.shared_paths(a, b, **kwargs)
/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.shared_paths(a, b, **kwargs)
/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.shared_paths(a, b, **kwargs)
/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.shared_paths(a, b, **kwargs)
/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.sha

(7, 4, 10, 3)

In [6]:
planner = cli.create_planner_from_config_file("benchmark/planner_config/CCRv2.yml", env)
planner.create_plan()

Plans(plans=[[7, 13, 7, 8, 3, 5, 12, 9, 4], [4, 4, 4, 9, 10, 13, 7], [10, 10, 10, 13, 7, 8, 3], [3, 5, 5, 5, 12, 9, 10]])

In [7]:
planner.history

[]

In [8]:
data = []
for i, plan in enumerate(planner.history[0]['solution']):
    print(plan)
    for t, n in enumerate(plan):
        #print(f"{t}: {n}")
        if n is None:
            continue
        x,y,*_ = env.g.nodes[n]['pos']
        data.append({"agent": i, "t": t, "node": n, "x": x, "y": y})
df = pd.DataFrame(data)
df.agent = df.agent.astype('category')
df.head(20)

IndexError: list index out of range

In [ ]:
import plotly.graph_objects as go


#df = pd.DataFrame(dict(x=[0], y=[0]))
fig = px.scatter(df, x="x", y="y", animation_frame='t', animation_group="agent", color="agent", hover_name="agent")
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')))
showlegend = True
#fig.update_layout(showlegend=False)
obs = env._obstacles.geoms
for o in obs:
    xy = o.exterior.xy
    fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(20,20,20,0.4)", name="Obstacles", showlegend=showlegend, mode="text"))
    showlegend = False
    
inner = [n['geometry'].inner for _, n in env.g.nodes(data=True)]
showlegend = True

for i in inner:
    xy = i.exterior.xy
    fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(0,100,80,0.2)", name="Inner", showlegend=showlegend, mode="text"))
    showlegend = False
    
inner = [n['geometry'].borderPoly for *_, n in env.g.edges(data=True)]
showlegend = True
for i in inner:
    xy = i.exterior.xy
    fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(0,100,40,0.05)", name="Transition", showlegend=showlegend, mode="text"))
    showlegend = False
fig.update_layout(width=1000, height=800)
fig.show()

NameError: name 'df' is not defined

In [9]:
from polygonal_roadmaps.cli import aggregate_results

df = aggregate_results("results")
df

,astar,failed,flowtime,spacetime_astar,steps,sum_of_cost,history,config.index,config.planner,config.planner_file,config.scen,config.xindex,config.planner_args.priority_method,config.planner_args.max_iter
0,0,False,12.0,0,12,58.0,results/PRIO.yml/DrivingSwarm;icra2024.yaml;ic...,0,PrioritizedPlanner,PRIO.yml,DrivingSwarm;icra2024.yaml;icra2024.yml,0,NaN,NaN
1,0,False,12.0,0,12,58.0,results/PRIO.yml/DrivingSwarm;icra2024.yaml;ic...,2,PrioritizedPlanner,PRIO.yml,DrivingSwarm;icra2024.yaml;icra2024.yml,0,NaN,NaN
2,0,False,12.0,0,12,57.0,results/PRIO.yml/DrivingSwarm;icra2024.yaml;ic...,4,PrioritizedPlanner,PRIO.yml,DrivingSwarm;icra2024.yaml;icra2024.yml,0,NaN,NaN
3,0,False,13.0,0,13,59.0,results/PRIO.yml/DrivingSwarm;icra2024.yaml;ic...,3,PrioritizedPlanner,PRIO.yml,DrivingSwarm;icra2024.yaml;icra2024.yml,0,NaN,NaN
4,0,False,13.0,0,13,59.0,results/PRIO.yml/DrivingSwarm;icra2024.yaml;ic...,1,PrioritizedPlanner,PRIO.yml,DrivingSwarm;icra2024.yaml;icra2024.yml,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0,False,1000.0,0,1000,3015.0,results/PrioIndex.yml/DrivingSwarm;icra2024.ya...,4,PriorityAgentPlanner,PrioIndex.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,index,NaN
64,0,False,2.0,0,2,10.0,results/PrioIndex.yml/DrivingSwarm;icra2024.ya...,3,PriorityAgentPlanner,PrioIndex.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,index,NaN
65,0,False,2.0,0,2,10.0,results/PrioIndex.yml/DrivingSwarm;icra2024.ya...,1,PriorityAgentPlanner,PrioIndex.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,index,NaN
66,0,False,5.0,0,5,12.0,results/PrioIndex.yml/DrivingSwarm;icra2021_ma...,1,PriorityAgentPlanner,PrioIndex.yml,DrivingSwarm;icra2021_map.yaml;icra2021.yml,0,index,NaN


In [11]:
df[df.steps.eq(0)]

,astar,failed,flowtime,spacetime_astar,steps,sum_of_cost,history,config.index,config.planner,config.planner_file,config.scen,config.xindex,config.planner_args.priority_method,config.planner_args.max_iter
6,0,False,0.0,0,0,inf,results/PRIO.yml/DrivingSwarm;icra2024.yaml;ic...,2,PrioritizedPlanner,PRIO.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,NaN,NaN
8,0,False,0.0,0,0,inf,results/PRIO.yml/DrivingSwarm;icra2024.yaml;ic...,3,PrioritizedPlanner,PRIO.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,NaN,NaN
15,0,True,0.0,0,0,inf,results/CCR.yml/DrivingSwarm;icra2024.yaml;icr...,0,CCR,CCR.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,NaN,NaN
22,0,False,0.0,0,0,inf,results/PrioSame.yml/DrivingSwarm;icra2024.yam...,0,PriorityAgentPlanner,PrioSame.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,same,NaN
23,0,False,0.0,0,0,inf,results/PrioSame.yml/DrivingSwarm;icra2024.yam...,2,PriorityAgentPlanner,PrioSame.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,same,NaN
24,0,False,0.0,0,0,inf,results/PrioSame.yml/DrivingSwarm;icra2024.yam...,4,PriorityAgentPlanner,PrioSame.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,same,NaN
25,0,False,0.0,0,0,inf,results/PrioSame.yml/DrivingSwarm;icra2024.yam...,3,PriorityAgentPlanner,PrioSame.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,same,NaN
26,0,False,0.0,0,0,inf,results/PrioSame.yml/DrivingSwarm;icra2024.yam...,1,PriorityAgentPlanner,PrioSame.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,same,NaN
35,0,False,0.0,0,0,inf,results/CCRv2.yml/DrivingSwarm;icra2024.yaml;i...,0,CCRv2,CCRv2.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,NaN,NaN
36,0,False,0.0,0,0,inf,results/CCRv2.yml/DrivingSwarm;icra2024.yaml;i...,2,CCRv2,CCRv2.yml,DrivingSwarm;icra2024.yaml;icra2024_1m.yml,0,NaN,NaN


In [10]:
df.history = df.history.astype(str).astype('category')
px.scatter(df.loc[~df.failed], x="config.planner", y="steps", color="config.planner", facet_col="config.scen", hover_name="history")

In [ ]:
def show_run(history_file, scenario):
    df = pd.read_feather(history_file)
    fig = px.scatter(df, x="x", y="y", animation_frame='t', animation_group="agent", color="agent", hover_name="agent")
    fig.update_traces(marker=dict(size=12,
                                  line=dict(width=2,
                                            color='DarkSlateGrey')))
    showlegend = True
#fig.update_layout(showlegend=False)
    env = cli.env_generator(scenario, n_agents=1)[0]
    obs = env._obstacles.geoms
    for o in obs:
        xy = o.exterior.xy
        fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(20,20,20,0.4)", name="Obstacles", showlegend=showlegend, mode="text"))
        showlegend = False
        
    inner = [n['geometry'].inner for _, n in env.g.nodes(data=True)]
    showlegend = True

    for i in inner:
        if i.geom_type != "Polygon":
            continue
        xy = i.exterior.xy
        fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(0,100,80,0.2)", name="Inner", showlegend=showlegend, mode="text"))
        showlegend = False
        
    inner = [n['geometry'].borderPoly for *_, n in env.g.edges(data=True)]
    showlegend = True
    for i in inner:
        if i.geom_type != "Polygon":
            continue
        xy = i.exterior.xy
        fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(0,100,40,0.05)", name="Transition", showlegend=showlegend, mode="text"))
        showlegend = False
    fig.update_layout(width=1000, height=800)
    fig.show()


#show_run(df.iloc[22]["history"], df.iloc[22]["config.scen"])
show_run("results/CCRv2.yml/DrivingSwarm;icra2024.yaml;icra2024.yml/1/history.feather", "DrivingSwarm;icra2024.yaml;icra2024.yml")

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarni

In [ ]:
df.iloc[22]

astar                                                                                  0
failed                                                                             False
flowtime                                                                             2.0
makespan                                                                               2
spacetime_astar                                                                        0
steps                                                                                  2
sum_of_cost                                                                           10
history                                results/CCRv2.yml/DrivingSwarm;icra2024.yaml;i...
config.index                                                                           1
config.planner                                                                     CCRv2
config.planner_file                                                            CCRv2.yml
config.scen          

In [ ]:
df.columns

Index(['astar', 'failed', 'flowtime', 'makespan', 'spacetime_astar', 'steps',
       'sum_of_cost', 'history', 'config.index', 'config.planner',
       'config.planner_file', 'config.scen', 'config.xindex',
       'config.planner_args.priority_method', 'config.planner_args.max_iter'],
      dtype='object')

In [ ]:
x = pd.read_feather(df.history.iloc[0])
x

,t,agent,state,x,y
0,0,0,9.0,-0.5,-1.5
1,0,1,1.0,2.5,1.0
2,0,2,0.0,2.5,-1.5
3,0,3,43.0,-1.0,1.0
4,0,4,2.0,2.5,-1.0
5,1,0,11.0,0.0,-1.5
6,1,1,44.0,2.0,1.0
7,1,2,2.0,2.5,-1.0
8,1,3,30.0,-1.0,0.5
9,1,4,36.0,2.0,-1.0
